In [3]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy
    

apple = 100
apple_num = 2 
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(f"price : {int(price)}")
print(f"dApple : {int(dapple)}")
print(f"dApple_num : {int(dapple_num)}")
print(f"dTax : {dtax}")

price : 220
dApple : 2
dApple_num : 110
dTax : 200


In [7]:
from common.layers import Affine
import numpy as np
x = np.array([[1,1]])
W = np.array([[1,2,3], [4,5,6]])
b = np.array([7,8,9])

지출 = Affine(W, b)
지출.forward(x)

지출.backward([[2,1,-1]])

array([[1, 7]])